Author: Vinícius Antoino Ramos Zecca

*LinkedIn: /viniciusarz*

*Github: /ViniciusARZ*


# Libraries and Access



In [ ]:
!pip install --upgrade gspread
!pip install pandas as pd
!pip install Numpy as np

## Google Authorization
---

In [ ]:
from google.colab import auth
auth.authenticate_user()

import pandas as pd
import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

## Workbook Access

Accessing Leads til 2020 File

In [ ]:
#Leads till 2020 file (Ploomes)

ws_funnel_2020 = gc.open_by_key('Key')
ploomes = ws_funnel_2020.worksheet('Ploomes')
rows_ploomes = ploomes.get_all_values()
ploomes_total_df = pd.DataFrame.from_records(rows_ploomes,columns=rows_ploomes[0])

In [ ]:
print(ploomes_total_df.columns)

Index(['cre_date', 'member', 'name', 'location', 'segment', 'mail', 'phone',
       'contact', 'contact_mail', 'last_date'],
      dtype='object')


Accessing Pipefy's Full file - Sales Funnel 2021

In [ ]:
# Pipefy's direct import to a spreadsheets

ws_pipefy_2021 = gc.open_by_key('Key')
# ATTENTION - This has to remain "Report"
pipefy = ws_pipefy_2021.worksheet('Report')
rows_pipefy = pipefy.get_all_values()
pipefy_total_df = pd.DataFrame.from_records(rows_pipefy,columns=rows_pipefy[0])

In [ ]:
print(pipefy_total_df.columns)

Index(['Created at', 'Assignees', 'Title', 'Setor', 'E-mail de Contato',
       'Origem', 'Telefone', 'Nome do Contato', 'Current phase',
       'Company size', 'Contato por:', 'Deal value ', 'Projetos',
       'Motivo Descarte', 'Reunião de Diagnóstico marcada', 'Dores do Cliente',
       'Haverá proposta?', 'Data da Reunião', 'Motivo Futuro',
       'First time enter Prospecção', 'Last time exit Prospecção',
       'Total time in Prospecção (days)', 'First time enter Descoberta',
       'Last time exit Descoberta', 'Total time in Descoberta (days)',
       'First time enter Reunião de Diagnóstico',
       'Last time exit Reunião de Diagnóstico',
       'Total time in Reunião de Diagnóstico (days)',
       'First time enter Reunião de Proposta',
       'Last time exit Reunião de Proposta',
       'Total time in Reunião de Proposta (days)',
       'First time enter Negociação', 'Last time exit Negociação',
       'Total time in Negociação (days)',
       'First time enter Contato para 

Acessing File that will receive the total Sales Database

In [ ]:
# Receiving File - Sales_Leads_Data_2018-2021

ws_totalsales = gc.open_by_key('Key')
# ATTENTION - This has to remain "Leads Data"
total_sales = ws_totalsales.worksheet('Leads Data')

Accessing File that will receive the Clead-up Pipefy Data

In [ ]:
# Receiving File - (DATASTUDIO) leads_2021_database

ws_datastudio_sheets = gc.open_by_key('key')
# ATTENTION - This has to remain "python update"
datastudio_sheets = ws_datastudio_sheets.worksheet('python update')
rows_dssheets = datastudio_sheets.get_all_values()
ds_df = pd.DataFrame.from_records(rows_dssheets,columns=rows_dssheets[0])

# Update Leads Database

In [ ]:
#Applying filter in pipefy's DF to make it work with Ploomes
# Ploomes' Columns -> Index(['cre_date', 'member', 'name', 'location', 'segment', 'mail', 'phone', 'contact', 'contact_mail', 'last_date']
filtered_database_pipefy = pipefy_total_df[["Created at", "Creator", "Nome da Empresa", "Setor", "E-mail de Contato","Telefone", "Nome do Contato", "E-mail do Contato ", "Updated at"]]
#ft_pipefy_df = filtered_database_pipefy.rename(columns={"Created at": "cre_date", "Creator": "member", "Nome da Empresa": "name", "Sector": "segment",
                                                        #"E-mail de Contato": "mail","Telefone": "phone", "Nome do Contato": "contact",
                                                        #"E-mail do Contato": "contact_mail", "Updated at": "last_date"}, inplace=True)
print(filtered_database_pipefy.columns)

In [ ]:
#Applying filter in Ploomes' DF to make it work with Pipefy
filtered_database_ploomes = ploomes_total_df[['cre_date', 'member', 'name', 'segment', 'mail', 'phone', 'contact', 'contact_mail', 'last_date']]
print(filtered_database_ploomes.columns)

Index(['cre_date', 'member', 'name', 'segment', 'mail', 'phone', 'contact',
       'contact_mail', 'last_date'],
      dtype='object')


Concatening the two dataframes in one for the Leads_Database

In [ ]:
filtered_database_pipefy.columns = filtered_database_ploomes.columns
leads_database = filtered_database_ploomes.append(filtered_database_pipefy, ignore_index = True)
print(leads_database.columns)

Index(['cre_date', 'member', 'name', 'segment', 'mail', 'phone', 'contact',
       'contact_mail', 'last_date'],
      dtype='object')


**Update Command**

In [ ]:
listconv_leadsdatabase = [leads_database.columns.tolist()] + leads_database.to_numpy().tolist()
total_sales.update('A1',listconv_leadsdatabase)

{'spreadsheetId': '1BYLzpjk5P8RdCE4PjlncnqQ2fSskQWbi_3l2ER2e2_Q',
 'updatedCells': 5616,
 'updatedColumns': 9,
 'updatedRange': "'Leads Data'!A1:I624",
 'updatedRows': 624}

#Clean-up Pipefy's Data and upload into the file that supports Data Studio

In [ ]:
#Final Columns:
#Created at	Assignees	Title	Setor	E-mail de Contato	Origem	Telefone	Nome do Contato	Current phase	Company size	Contato por:	
#Deal value 	Projetos	Motivo Descarte	Reunião Marcada	Dores do Cliente	Haverá proposta?	Data da Reunião	Motivo Futuro	FTE 
#Prospecção	Last time exit Prospecção	Total time in Prospecção (days)	FTE Descoberta	Last time exit Descoberta	Total time in Descoberta (days)	
#FTE Reunião de Diagnóstico	Last time exit Reunião de Diagnóstico	Total time in Reunião de Diagnóstico (days)	FTE Reunião de Proposta	
#Last time exit Reunião de Proposta	Total time in Reunião de Proposta (days)	FTE Negociação	Last time exit Negociação	
#Total time in Negociação (days)	FTE Contato para o Futuro	Last time exit Contato para o Futuro	
#Total time in Contato para o Futuro (days)	FTE GANHAMOOO 	FTE perdemo... 	Last time exit perdemo... 	Total time in perdemo...  (days)

In [ ]:
pipefy_total_df.rename(columns={'First time enter Prospecção': 'FTE Prospecção','First time enter Descoberta':'FTE Descoberta',
                                             'First time enter Reunião de Diagnóstico': 'FTE Reunião de Diagnóstico',
                                             'First time enter Reunião de Proposta': 'FTE Reunião de Proposta','First time enter Negociação': 'FTE Negociação',
                                             'First time enter Contato para o Futuro': 'FTE Contato para o Futuro',
                                             'First time enter GANHAMOOO 😍': 'FTE Ganho','First time enter perdemo... 😞': 'FTE Perdido'}, inplace=True)

In [ ]:
print(pipefy_total_df.columns)

Index(['Created at', 'Assignees', 'Title', 'Setor', 'E-mail de Contato',
       'Origem', 'Telefone', 'Nome do Contato', 'Current phase',
       'Company size', 'Contato por:', 'Deal value ', 'Projetos',
       'Motivo Descarte', 'Reunião de Diagnóstico marcada', 'Dores do Cliente',
       'Haverá proposta?', 'Data da Reunião', 'Motivo Futuro',
       'FTE Prospecção', 'Last time exit Prospecção',
       'Total time in Prospecção (days)', 'FTE Descoberta',
       'Last time exit Descoberta', 'Total time in Descoberta (days)',
       'FTE Reunião de Diagnóstico', 'Last time exit Reunião de Diagnóstico',
       'Total time in Reunião de Diagnóstico (days)',
       'FTE Reunião de Proposta', 'Last time exit Reunião de Proposta',
       'Total time in Reunião de Proposta (days)', 'FTE Negociação',
       'Last time exit Negociação', 'Total time in Negociação (days)',
       'FTE Contato para o Futuro', 'Last time exit Contato para o Futuro',
       'Total time in Contato para o Futuro (day

In [ ]:
dstudio_df_cleaned = pipefy_total_df[['Created at',	'Assignees',	'Title',	'Setor',	'E-mail de Contato',	'Origem',	'Telefone',	'Nome do Contato',
                                      'Current phase',	'Company size',	'Contato por:',	'Deal value ', 	'Projetos',	'Motivo Descarte',	'Reunião de Diagnóstico marcada',	'Dores do Cliente',
                                      'Haverá proposta?',	'Data da Reunião',	'Motivo Futuro',	'FTE Prospecção',	'Total time in Prospecção (days)',	'FTE Descoberta',
                                      'Total time in Descoberta (days)',	'FTE Reunião de Diagnóstico',	'Total time in Reunião de Diagnóstico (days)',
                                      'FTE Reunião de Proposta', 'Total time in Reunião de Proposta (days)',	'FTE Negociação', 'Total time in Negociação (days)',
                                      'FTE Contato para o Futuro',	'Total time in Contato para o Futuro (days)',	'FTE Ganho', 'FTE Perdido', 'Total time in perdemo... 😞 (days)']]

In [ ]:
dstudio_pipefy_database = [dstudio_df_cleaned.columns.tolist()] + dstudio_df_cleaned.to_numpy().tolist()
datastudio_sheets.update('A1',dstudio_pipefy_database)

{'spreadsheetId': '1yIixYpMxuY8ftFZdt3zFIljdSxz5BhxChKgCIVdVguc',
 'updatedCells': 9622,
 'updatedColumns': 34,
 'updatedRange': "'python update'!A1:AH283",
 'updatedRows': 283}